In [1]:
from SOM import SOM
import sys

sys.path.append("../../")
from utils import load_csv, dataloader, load_save_models
import torch
from torchvision.transforms import v2
import preprocessing.tiff_handling as tiff_handling
import plotly.express as px
from stats import evaluate_perf_utils
import numpy as np

## load in the data

In [2]:
input_dim = 100
map_dim = 80

In [3]:
model = SOM.load_model(SOM(input_dim=input_dim,map_dim=map_dim),'./hpc_SOM_model.pkl')
model.som

Model loaded from ./hpc_SOM_model.pkl.


## load in data

In [4]:
set = load_csv.load_pandas()
train, val,test = load_csv.split_data(set)

print(f'outlier test validation: {any(val["classification"]!=1)}')

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/SOM/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


outlier test validation: False


In [5]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        # v2.Lambda(
        #     lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        # ),
    ]
)

val_data = np.array(
    [
        tiff_handling.flatten_image(transform(tiff_handling.handle_tiff(el)))
        for el in val["file_names"].tolist()
    ]
)

test["image"] = test["file_names"].apply(lambda x: tiff_handling.flatten_image(transform(tiff_handling.handle_tiff(x))))

In [12]:
# optimize percentile of error threshold
import optuna

truth = [1 if value == 1 else 0 for value in test["classification"]]


def objective(trial):
    perc = trial.suggest_int("percentile", 80,100,step=5)
    model.set_threshold(val_data,perc)
    output = model.predict(test)

    perf_metrics = evaluate_perf_utils.evaluate_performance(truth, output["prediction"])
    return perf_metrics["f1"]


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

[I 2024-12-03 22:05:48,172] A new study created in memory with name: no-name-d5b68005-1f29-4b4c-bde8-25698b6ad886


Threshold set at 2.8702 (Percentile: 95)


[I 2024-12-03 22:17:54,493] Trial 0 finished with value: 0.7945439045183291 and parameters: {'percentile': 95}. Best is trial 0 with value: 0.7945439045183291.


Threshold set at 2.3000 (Percentile: 80)


[I 2024-12-03 22:30:14,541] Trial 1 finished with value: 0.8269364968597348 and parameters: {'percentile': 80}. Best is trial 1 with value: 0.8269364968597348.


...

[I 2024-12-03 22:42:11,089] Trial 2 finished with value: 0.6695772058823529 and parameters: {'percentile': 100}. Best is trial 1 with value: 0.8269364968597348.


Threshold set at 2.4062 (Percentile: 85)


[I 2024-12-03 22:54:36,556] Trial 3 finished with value: 0.832 and parameters: {'percentile': 85}. Best is trial 3 with value: 0.832.


Threshold set at 2.5560 (Percentile: 90)


[I 2024-12-03 23:07:05,579] Trial 4 finished with value: 0.8363522798251093 and parameters: {'percentile': 90}. Best is trial 4 with value: 0.8363522798251093.


In [13]:
print(f"best percentile: {study.best_params['percentile']}")
print(f"best f1: {study.best_value}")

best percentile: 90
best f1: 0.8363522798251093


In [14]:
model.set_threshold(val_data,percentile=90)

Threshold set at 2.5560 (Percentile: 90)


np.float64(2.5559949231468084)

In [15]:
output = model.predict(test)

In [16]:
truth = [1 if value == 1 else 0 for value in output['classification']]

perf_metrics = evaluate_perf_utils.evaluate_performance(truth, output['prediction'])
tn = perf_metrics['tn']
tp = perf_metrics['tp']
fn = perf_metrics['fn']
fp = perf_metrics['fp']
precision = perf_metrics['precision']
recall = perf_metrics['recall']
f1 = perf_metrics['f1']

perf_metrics

{'tn': np.int64(1051),
 'fp': np.int64(406),
 'fn': np.int64(118),
 'tp': np.int64(1339),
 'precision': np.float64(0.7673352435530086),
 'recall': np.float64(0.9190116678105696),
 'f1': np.float64(0.8363522798251093)}

In [17]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f'accuracy: {accuracy}')

accuracy: 0.820178448867536
